In [171]:
import pandas as pd
import numpy as np
import freeman as fm
import networkx as nx
from collections import defaultdict as ddict
from typing import DefaultDict, Set

In [136]:
bruno = pd.read_csv(
    "csv/players.csv",
     usecols=("date", "player_name", "team", "player_id", "match_id", "event_id", "rating"),
)

In [ ]:
bruno.loc[:, "date"] = pd.to_datetime(bruno["date"], format="%Y-%m-%d")
bruno.loc[:, "player_name"] = bruno["player_name"].apply(
    lambda x: str(x).encode('ascii', 'backslashreplace').decode('ascii')
)

In [137]:
bruno.cabeca = bruno.head
bruno.cabeca()
# type(bruno["player_name"].iloc[0])
# bruno[bruno["player_name"].apply(lambda x: type(x)!= str)]
# _ = bruno["player_name"].apply(lambda x: print(x) if type(x) != str else None)

,date,player_name,team,player_id,match_id,event_id,rating
0,2020-02-26,Brehze,Evil Geniuses,9136,2339385,4901,1.04
1,2020-02-26,CeRq,Evil Geniuses,11219,2339385,4901,0.98
2,2020-02-26,EliGE,Liquid,8738,2339385,4901,1.08
3,2020-02-26,Ethan,Evil Geniuses,10671,2339385,4901,0.83
4,2020-02-26,NAF,Liquid,8520,2339385,4901,1.08


In [138]:
upid_rating = bruno.groupby(["player_id", "player_name"])["rating"]
median = upid_rating.median()
median.name = "median"
mean = upid_rating.mean()
mean.name = "mean"

helionp = pd.concat([median, mean], axis=1)
helionp.ceara = helionp.head
helionp.ceara()

,,median,mean
player_id,player_name,,
2,RobbaN,0.775,0.775000
7,Friis,1.030,1.047344
11,Vertigo,0.925,0.925000
13,RashiE,1.240,1.201818
15,m1kkis,0.990,1.019756


In [174]:
teams: DefaultDict[str, Set[str]] = ddict(set)
_ = bruno.groupby(["team", "match_id"])["player_id"].apply(
    lambda x: teams[x.name[0]].update(x.values)
)

In [168]:
with open("csgo.gml", "w+") as f:
    f.write("graph [\n    directed 0\n")
    for (p_id, p_name), (median, mean) in helionp.iterrows():
        f.write("    node [\n")
        f.write(f"        id {p_id}\n")
        f.write(f"        label \"{p_name}\"\n")
        f.write("    ]\n")
    for team in teams:
        # FIXME duplicate edges
        players = iter(teams[team])
        first = next(players)

        for player in players:
            f.write("    edge [\n")
            f.write(f"        source {first}\n")
            f.write(f"        target {player}\n")
            f.write("    ]\n")


    f.write("]\n")

In [169]:
g = fm.load('csgo.gml')
g.set_all_nodes(size=10, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.25))
g.move("kamada_kawai")
g.draw()

NetworkXError: edge #933 (10282--10285) is duplicated

In [143]:
print(bruno["player_name"].unique().shape)
print(bruno["player_id"].unique().shape)

(11992,)
(12295,)


In [154]:
s = set()
for i in helionp.index.values:
#     print(i)
    if i[0] in s:
        print(i)
    else:
        s.add(i[0])
#     if helionp.loc[i[0]].shape[0] != 1:
#         print(i)